In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import torch
from tqdm.auto import tqdm

from transformers import AutoModel, AutoTokenizer

ModuleNotFoundError: No module named 'torch'

In [ ]:
# Import
PATH = 'drive/MyDrive/SIE533/Project/'
content = pd.read_csv(f'{PATH}/content_filtered.csv')
correlation = pd.read_csv(f'{PATH}/correlation.csv')
topics = pd.read_csv(f'{PATH}/topics_filtered.csv')

In [ ]:
content

In [ ]:
correlation

In [ ]:
topics

In [ ]:
# Model Import
MODEL = 'sentence-transformers/paraphrase-MiniLM-L12-v2'
device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = AutoModel.from_pretrained(MODEL)
model.eval()
model.to(device)

tokenizer = AutoTokenizer.from_pretrained(MODEL)

In [ ]:

vecs = []
MAX_LEN = 384
# ELEMENT = content['title']

for _,row in tqdm(content.iterrows(), total=len(content)):
  # print(content['title'][row])
  title = row['title']
  if type(title) is float:
    title = row['description']
  if type(title) is float:
    title = row['text']

  tok = tokenizer(title)
  for k,v in tok.items():
    tok[k] = torch.tensor(v[:MAX_LEN]).to(device).unsqueeze(0)
  with torch.no_grad():
    output = model(**tok)

  vec = output.last_hidden_state.squeeze(0).mean(0).cpu()
  vecs.append(vec)

vecs1 = torch.stack(vecs)